<h1><center>619 Analysis</center></h1>

The analyses below are the final (or final until I get feedback from the PIs) analysis for my 619 data. I have modeled this analysis after Greening & Mitchell, 2015 in Human Brain Mapping. In this analysis, we are using a machine learning permutation test of a Ridge Regression model with the probability of connectivity of the amygdala with eight Brodmann's Areas located in the prefrontal cortex. After that, we used 1000 iterations of a training and test dataset where a unique 80% of the data is allocted to the training dataset and a unique 20% to the test dataset to determine which coefficients were reliably contributing to the model. Here's what that looks likes for the left and right hemispheres. 

In [1]:
# This is necessary for the plots to print in the jupyter notebook here.
%matplotlib inline
# Import the necessary packages (double check to make sure that all of these are used).
import numpy as np
import scipy as sp
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import csv

from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import permutation_test_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

In [ ]:
import sys
print(sys.version)

In [ ]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
# Import the data. 
#filename = '/Users/leighgayle/Box Sync/Final_619_Docs/ProbtrackData_NoScalpSubList_wDti_Final_120117.csv'
filename = '/Users/leighgayle/Box Sync/Final_619_Docs/ProbtrackData_NoScalpSubList_wDti_Final_103018.csv'
#filename = '/Users/leighgayle/Box Sync/ThreatDep_Probtrack_Amygdala/Amy_NewProbtrack_nomissing.csv'
data = pd.read_csv(filename, header=0)

I'm going to remove the outliers in the data now that are calculated in R because it seems like a giant pain to figure out how to do it in python when I already know how to calculate influential outliers in R. Fun fact for everyone in the room--R is 1 index and python is 0 index. The outiers detected in R were 12, 75, 82, 89, 99, and 146. That would make them 11, 74, 81, 88, 98, and 145 in python. I'm taking them out in reverse so It doesn't mess with the earlier indexing.

The subjects removed are 10023, 10116, 10125, 10136, 10155, and 10234.

These outliers are removed based on Cook's Distance of the linear regression models with the cutoff being 4/(n-k-1) or 0.029.

In [ ]:
# Remove outliers detected in our R dataset.  
#clean_data = data.drop(145)
clean_data = data.drop(144)

In [ ]:
clean_data = clean_data.drop(98)

In [ ]:
clean_data = clean_data.drop(88)

In [ ]:
clean_data = clean_data.drop(81)

In [ ]:
clean_data = clean_data.drop(74)

In [ ]:
clean_data = clean_data.drop(11)

In [ ]:
clean_data.shape

In [ ]:
clean_data

In [ ]:
# Check to make sure the subjects that we want to be deleted are deleted--success.
clean_data.to_csv(path_or_buf='clean_data_output_final_052818.csv',sep=',')

In [ ]:
# Scale the data -- this should be done in Ridge Regression
#scaler = StandardScaler()
#scaler.fit(clean_data[["Threat_Act_Ramy_NoScalp","RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target", "RAmySeed_BA46Target","RAmySeed_BA11Target","RAmySeed_BA47Target","Threat_Act_Lamy_NoScalp","LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target", "LAmySeed_BA46Target","LAmySeed_BA11Target","LAmySeed_BA47Target"]])
#StandardScaler(copy=True, with_mean=True, with_std=True)
#scaled_data = scaler.transform(clean_data[["Threat_Act_Ramy_NoScalp","RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target", "RAmySeed_BA46Target","RAmySeed_BA11Target","RAmySeed_BA47Target","Threat_Act_Lamy_NoScalp","LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target", "LAmySeed_BA46Target","LAmySeed_BA11Target","LAmySeed_BA47Target"]])

# Actually, normalize option will take care of this. I will keep this in here in case I would like to switch it back eventually, but I think how we have the data currently is sufficient. 
# Similarly, I'll keep the scaled data options commented out below, I do not use them at the current moment though. 

In [ ]:
# specify the X and y as scaled variables
y_r = clean_data.Threat_Act_Ramy_NoScalp
X_r = clean_data[["RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target","RAmySeed_BA11Target","RAmySeed_BA47Target"]]
#y_r = scaled_data[:,0]
#X_r = scaled_data[:,[1,2,3,4,5,7,8]]

y_l = clean_data.Threat_Act_Lamy_NoScalp
X_l = clean_data[["LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target","LAmySeed_BA11Target","LAmySeed_BA47Target"]]
#y_l = scaled_data[:,9]
#X_l = scaled_data[:,[10,11,12,13,14,16,17]]

Once we have the variables loaded in, it's time to have the RidgeCV command choose which regularization parameter fits the data. We'll look at it for both of our regression equations, but it ends up being the same. 

The command below is creating a ridge model where it's cycling through alphas between 1 and 10 with steps of 0.01. 

In [ ]:
# Pick which alpha to use. 
ridgeCV = linear_model.RidgeCV(alphas=[0.1, 0.001, 10.0], normalize=True)

In [ ]:
# Right
ridgeCV.fit(X_r,y_r)
print "The suggested regularization parameter for the right hemisphere regression is {}".format(ridgeCV.alpha_)

In [ ]:
# Left
ridgeCV.fit(X_l,y_l)
print "The suggested regularization parameter for the left hemisphere regression is {}".format(ridgeCV.alpha_)

In [ ]:
# Define our ridge regression model.
ridge = linear_model.Ridge(alpha=0.1, normalize = True)

<h3>Right Hemisphere Model Test</h3>

In [ ]:
#score_R, permuatation_scores_R, pvalue_R = permutation_test_score(ridge, X_r, y_r, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)
score_R, permuatation_scores_R, pvalue_R = permutation_test_score(ridge, X_r, y_r, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
score_R

In [ ]:
pvalue_R

In [ ]:
with open('clean_r_hemisphere_permutation_scores_final_103018.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    listwriter.writerow(permuatation_scores_R)

In [ ]:
# Plot the permutations.
#tnr_font = {'fontname':'Times New Roman'}

from mpl_toolkits.axes_grid.axislines import Subplot

plt.rcParams["font.family"] = "Times New Roman"
fig = plt.figure()
fig.patch.set_facecolor('white')

ax = Subplot(fig, 111)
fig.add_subplot(ax)

ax.axis["right"].set_visible(False)
ax.axis["top"].set_visible(False)

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

n_classes = np.unique(y_r).size
plt.hist(permuatation_scores_R*-1, 20, label='Permutation MSE Scores',
         edgecolor='black',color='grey')
ylim = plt.ylim()
plt.plot(2 * [score_R*-1], ylim, '--g', linewidth=3,
         label='Predicted MSE Score'
         ' (p-value %.3f)' % pvalue_R, color='k')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score', fontsize='large')
plt.ylabel('Number of Permutations', fontsize='large')

#plt.title(r"${"+ str('Figure' '\ XX:')+"}$" + 'Threat Amygdala Activation (Right) Predicted by Prefrontal Probabilistic Tractography', y=-.3, fontsize='medium')

plt.show()

In [ ]:
# Let's look at the individual coefficients now.
# configure bootstrap
#clean_vals = clean_data.values
clean_vals = clean_data.values
n_iterations = 1000
n_size = int(len(clean_vals) * 0.84)
# run bootstrap
#with open('r_hemisphere_coefficients_newprobtrack_072318.csv', 'w') as csvfile:
#    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#    for i in range(n_iterations):
#        # prepare train and test sets
#        train = resample(clean_vals, n_samples=n_size)
#        test = np.array([x for x in clean_vals if x.tolist() not in train.tolist()])
#        # fit model
#        ridge.fit(train[:,[23,25,27,29,31,35,37,4]], train[:,15])
#        # evaluate model
#        predictions = ridge.predict(test[:,[23,25,27,29,31,35,37,4]])
#        listwriter.writerow(ridge.coef_)
        

# run bootstrap
with open('clean_r_hemisphere_coefficients_final_110818.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(clean_vals, n_samples=n_size)
        test = np.array([x for x in clean_vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23,25,27,29,31,35,37]], train[:,15])
        # evaluate model
        predictions = ridge.predict(test[:,[23,25,27,29,31,35,37]])
        listwriter.writerow(ridge.coef_)
    
    
#    with open('r_hemisphere_coefficients_110818.csv', 'w') as csvfile:
#    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
#    for i in range(n_iterations):
#        # prepare train and test sets
#        train = resample(clean_vals, n_samples=n_size)
#        test = np.array([x for x in clean_vals if x.tolist() not in train.tolist()])
#        # fit model
#        ridge.fit(train[:,[16,17,18,19,20,21,22]], train[:,8])
#        # evaluate model
#        predictions = ridge.predict(test[:,[16,17,18,19,20,21,22]])
#        listwriter.writerow(ridge.coef_)

In [ ]:
ridge.fit(X_r,y_r)

In [ ]:
with open('predicted_r_vals_final_103018.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    listwriter.writerow(ridge.predict(X_r))

<h3>Left Hemisphere Model Test</h3>

In [ ]:
score_L, permuatation_scores_L, pvalue_L = permutation_test_score(ridge, X_l, y_l, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
score_L

In [ ]:
pvalue_L

In [ ]:
with open('clean_l_hemisphere_permutation_scores_final_050718.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    listwriter.writerow(permuatation_scores_L)

In [ ]:
# Plot the permutations.
fig = plt.figure()
fig.patch.set_facecolor('white')

ax = Subplot(fig, 111)
fig.add_subplot(ax)

ax.axis["right"].set_visible(False)
ax.axis["top"].set_visible(False)

plt.hist(permuatation_scores_L*-1, 20, label='Permutation MSE Scores',
         edgecolor='black', color='grey')
ylim = plt.ylim()
plt.plot(2 * [score_L*-1], ylim, '--g', linewidth=3,
         label='Predicted MSE Score'
         ' (p-value %.3f)' % pvalue_L, color='k')

plt.ylim(ylim)
plt.legend()
plt.xlabel('Score', fontsize='large')
plt.ylabel('Number of Permutations', fontsize='large')
plt.title(r"${"+ str('Figure' '\ XX:')+"}$" + 'Threat Amygdala Activation (Left) Predicted by Prefrontal Probabilistic Tractography', y=-.3, fontsize=MEDIUM_SIZE)
plt.show()

In [ ]:
# run bootstrap
clean_vals = clean_data.values
n_iterations = 1000
n_size = int(len(clean_vals) * 0.84)
with open('clean_l_hemisphere_coefficients_final_110818.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(clean_vals, n_samples=n_size)
        test = np.array([x for x in clean_vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[22,24,26,28,30,34,36]], train[:,16])
        # evaluate model
        predictions = ridge.predict(test[:,[22,24,26,28,30,34,36]])
        listwriter.writerow(ridge.coef_)
        
        

In [ ]:
ridge.fit(X_l,y_l)
with open('predicted_l_vals_final_103018.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    listwriter.writerow(ridge.predict(X_l))

The predicted test scores were saved out as csv files. I then rank ordered the values and all of the predicted values and the coefficients that were reliably less than 0, meaning that more than 950 had coefficients less than 0, I consider to contribute reliably. 

Quick look added 5.24.2018 to look at all Faces models.

In [ ]:
score_LAF, permuatation_scores_LAF, pvalue_LAF = permutation_test_score(ridge, X_l, y_l_af, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)



In [ ]:
score_LAF

In [ ]:
pvalue_LAF

In [ ]:
score_RAF, permuatation_scores_RAF, pvalue_RAF = permutation_test_score(ridge, X_r, y_r_af, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)



In [ ]:
score_RAF

In [ ]:
pvalue_RAF

# Analyses done post R&R
## Analyses done 1.7.2019 - 1.11.2019

In [ ]:
filename = '/Users/leighgayle/Box Sync/Final_619_Docs/NeuroImage_Submission/R&R/clean_updated_datafile.csv'

In [ ]:
data = pd.read_csv(filename, header=0)

In [ ]:
y_r = data.Threat_Act_Ramy_NoScalp
y_r_happy = data.happyGTbaseline_right
y_r_sad = data.sadGTbaseline_right
X_r = data[["RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target","RAmySeed_BA11Target","RAmySeed_BA47Target","puberty_score","Gender","Internalizing"]]

y_l = data.Threat_Act_Lamy_NoScalp
y_l_happy = data.happyGTbaseline_left
y_l_sad = data.sadGTbaseline_left
X_l = data[["LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target","LAmySeed_BA11Target","LAmySeed_BA47Target","puberty_score","Gender","Internalizing"]]



In [6]:
# Define our ridge regression model.
ridge = linear_model.Ridge(alpha=0.1, normalize = True)

In [ ]:
score_R, permuatation_scores_R, pvalue_R = permutation_test_score(ridge, X_r, y_r, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_R

In [ ]:
score_L, permuatation_scores_L, pvalue_L = permutation_test_score(ridge, X_l, y_l, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_L

In [ ]:
vals = data.values
n_iterations = 1000
n_size = int(len(vals) * 0.84)


In [ ]:
# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariates_010819.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals, n_samples=n_size)
        test = np.array([x for x in vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]], train[:,16])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]])
        listwriter.writerow(ridge.coef_)

In [ ]:
# run bootstrap
with open('clean_l_hemisphere_coefficients_wcovariates_010819.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals, n_samples=n_size)
        test = np.array([x for x in vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45]], train[:,17])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45]])
        listwriter.writerow(ridge.coef_)

#### Incorporate Income

In [ ]:
filename2 = '/Users/leighgayle/Box Sync/Final_619_Docs/NeuroImage_Submission/R&R/clean_updated_datafile_nomissingincome.csv'

In [ ]:
data2 = pd.read_csv(filename2, header=0)

In [ ]:
y_r_ses = data2.Threat_Act_Ramy_NoScalp
X_r_ses = data2[["RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target","RAmySeed_BA11Target","RAmySeed_BA47Target","puberty_score","Gender","Internalizing","yr_income"]]

y_l_ses = data2.Threat_Act_Lamy_NoScalp
X_l_ses = data2[["LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target","LAmySeed_BA11Target","LAmySeed_BA47Target","puberty_score","Gender","Internalizing","yr_income"]]



In [ ]:
score_R_SES, permuatation_scores_R_SES, pvalue_R_SES = permutation_test_score(ridge, X_r_ses, y_r_ses, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_R_SES

In [ ]:
score_R_SES

In [ ]:
score_L_SES, permuatation_scores_L_SES, pvalue_L_SES = permutation_test_score(ridge, X_l_ses, y_l_ses, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_L_SES

In [ ]:
score_L_SES

In [ ]:
vals2 = data2.values
n_iterations = 1000
n_size = int(len(vals2) * 0.84)

# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariatesincSES_011019.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals2, n_samples=n_size)
        test = np.array([x for x in vals2 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45, 43]], train[:,16])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45, 43]])
        listwriter.writerow(ridge.coef_)

In [ ]:
# run bootstrap
with open('clean_l_hemisphere_coefficients_wcovariatesincSES_011019.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals2, n_samples=n_size)
        test = np.array([x for x in vals2 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45, 43]], train[:,17])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45, 43]])
        listwriter.writerow(ridge.coef_)

#### Test with other emotions

In [ ]:
score_R_h, permuatation_scores_R_h, pvalue_R_h = permutation_test_score(ridge, X_r, y_r_happy, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)



In [ ]:
pvalue_R_h

In [ ]:
vals = data.values
n_iterations = 1000
n_size = int(len(vals) * 0.84)

# run bootstrap
with open('clean_r_hemisphere_coefficients_happy_wcovariates_011019.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals, n_samples=n_size)
        test = np.array([x for x in vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]], train[:,47])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]])
        listwriter.writerow(ridge.coef_)

In [ ]:
score_L_h, permuatation_scores_L_h, pvalue_L_h = permutation_test_score(ridge, X_l, y_l_happy, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_L_h

In [ ]:
# run bootstrap
with open('clean_l_hemisphere_coefficients_happy_wcovariates_011019.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals, n_samples=n_size)
        test = np.array([x for x in vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45]], train[:,46])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 45]])
        listwriter.writerow(ridge.coef_)

In [ ]:
score_R_s, permuatation_scores_R_s, pvalue_R_s = permutation_test_score(ridge, X_r, y_r_sad, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_R_s

In [ ]:
with open('clean_r_hemisphere_coefficients_sad_wcovariatesincSES_011019.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals, n_samples=n_size)
        test = np.array([x for x in vals if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]], train[:,49])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 45]])
        listwriter.writerow(ridge.coef_)

In [ ]:
score_L_s, permuatation_scores_L_s, pvalue_L_s = permutation_test_score(ridge, X_l, y_l_sad, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)

In [ ]:
pvalue_L_s

#### MFQ & SCARED rather than Internalizing Factor

In [2]:
filename3 = '/Users/leighgayle/Box Sync/Final_619_Docs/NeuroImage_Submission/R&R/clean_data_wMFQSCARED.csv'


In [3]:
data3 = pd.read_csv(filename3, header=0)

In [4]:
y_r = data3.Threat_Act_Ramy_NoScalp
y_r_happy = data3.happyGTbaseline_right
y_r_sad = data3.sadGTbaseline_right
X_r_sym = data3[["RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target","RAmySeed_BA11Target","RAmySeed_BA47Target","puberty_score","Gender","MFQ_C_Sum","SCARED_C_Sum"]]

y_l = data3.Threat_Act_Lamy_NoScalp
y_l_happy = data3.happyGTbaseline_left
y_l_sad = data3.sadGTbaseline_left
X_l_sym = data3[["LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target","LAmySeed_BA11Target","LAmySeed_BA47Target","puberty_score","Gender","MFQ_C_Sum","SCARED_C_Sum"]]



In [8]:
score_R_sym, permuatation_scores_R_sym, pvalue_R_sym = permutation_test_score(ridge, X_r_sym, y_r, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [9]:
pvalue_R_sym

0.001999600079984003

In [10]:
score_R_sym

-0.33856497019967197

In [ ]:
vals3 = data3.values
n_iterations = 1000
n_size = int(len(vals3) * 0.84)

# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariates_MFQ_SCARED_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals3, n_samples=n_size)
        test = np.array([x for x in vals3 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]], train[:,16])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]])
        listwriter.writerow(ridge.coef_)

In [11]:
score_L_sym, permuatation_scores_L_sym, pvalue_L_sym = permutation_test_score(ridge, X_l_sym, y_l, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [12]:
pvalue_L_sym

0.013797240551889621

In [13]:
score_L_sym

-0.2922146206873778

In [ ]:
vals3 = data3.values
n_iterations = 1000
n_size = int(len(vals3) * 0.84)

# run bootstrap
with open('clean_l_hemisphere_coefficients_wcovariates_MFQ_SCARED_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals3, n_samples=n_size)
        test = np.array([x for x in vals3 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55]], train[:,17])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55]])
        listwriter.writerow(ridge.coef_)

#### Re-testing models with income & MFQ/SCARED rather than the internalizing factor score.

In [ ]:
filename4 = '/Users/leighgayle/Box Sync/Final_619_Docs/NeuroImage_Submission/R&R/clean_data_wMFQSCARED_nomissingincome.csv'


In [ ]:
data4 = pd.read_csv(filename4, header=0)

In [ ]:
y_r = data4.Threat_Act_Ramy_NoScalp
X_r_sym_ses = data4[["RAmySeed_BA25Target","RAmySeed_BA24Target","RAmySeed_BA32Target","RAmySeed_BA10Target", "RAmySeed_BA9Target","RAmySeed_BA11Target","RAmySeed_BA47Target","puberty_score","Gender","MFQ_C_Sum","SCARED_C_Sum","yr_income"]]

y_l = data4.Threat_Act_Lamy_NoScalp
X_l_sym_ses = data4[["LAmySeed_BA25Target","LAmySeed_BA24Target","LAmySeed_BA32Target","LAmySeed_BA10Target", "LAmySeed_BA9Target","LAmySeed_BA11Target","LAmySeed_BA47Target","puberty_score","Gender","MFQ_C_Sum","SCARED_C_Sum","yr_income"]]



In [ ]:
score_R_sym_ses, permuatation_scores_R_sym_ses, pvalue_R_sym_ses = permutation_test_score(ridge, X_r_sym_ses, y_r, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [ ]:
pvalue_R_sym_ses

In [ ]:
vals4 = data4.values
n_iterations = 1000
n_size = int(len(vals4) * 0.84)

# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariates_MFQ_SCARED_income_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals4, n_samples=n_size)
        test = np.array([x for x in vals4 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55, 43]], train[:,16])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55, 43]])
        listwriter.writerow(ridge.coef_)

Same as previously -- income is not a significant predictor, but we lose BA10 as a significant predictor. 

In [ ]:
score_L_sym_ses, permuatation_scores_L_sym_ses, pvalue_L_sym_ses = permutation_test_score(ridge, X_l_sym_ses, y_l, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [ ]:
pvalue_L_sym_ses

In [ ]:
# run bootstrap
with open('clean_l_hemisphere_coefficients_wcovariates_MFQ_SCARED_income_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals4, n_samples=n_size)
        test = np.array([x for x in vals4 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55, 43]], train[:,17])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55, 43]])
        listwriter.writerow(ridge.coef_)

Same as when controlling for just internalizing factor score -- income is not a significant predictor, but we lose BA11 as a significant predictor. 

#### Re-testing the models with the other emotions controlling for regular covariates (not income). 

In [14]:
score_R_sym_h, permuatation_scores_R_sym_h, pvalue_R_sym_h = permutation_test_score(ridge, X_r_sym, y_r_happy, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [15]:
pvalue_R_sym_h

0.01899620075984803

In [16]:
score_R_sym_h

-0.22728981245140134

In [ ]:
# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariates_MFQ_SCARED_happy_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals3, n_samples=n_size)
        test = np.array([x for x in vals3 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]], train[:,47])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]])
        listwriter.writerow(ridge.coef_)

Same as with the internalizing factor score -- signifciant model with BA10 and BA47 as significant predictors. 

In [17]:
score_L_sym_h, permuatation_scores_L_sym_h, pvalue_L_sym_h = permutation_test_score(ridge, X_l_sym, y_l_happy, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [18]:
pvalue_L_sym_h

0.01259748050389922

In [19]:
score_L_sym_h

-0.1513684443468151

In [7]:
vals3 = data3.values
n_iterations = 1000
n_size = int(len(vals3) * 0.84)

# run bootstrap
with open('clean_l_hemisphere_coefficients_wcovariates_MFQ_SCARED_happy_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals3, n_samples=n_size)
        test = np.array([x for x in vals3 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55]], train[:,46])
        # evaluate model
        predictions = ridge.predict(test[:,[23, 25, 27, 29, 31, 35, 37, 2, 44, 54, 55]])
        listwriter.writerow(ridge.coef_)

Significant model with BA10 as the only tract that is a unique predictor of variance

In [20]:
score_R_sym_s, permuatation_scores_R_sym_s, pvalue_R_sym_s = permutation_test_score(ridge, X_r_sym, y_r_sad, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [21]:
pvalue_R_sym_s

0.006998600279944011

In [22]:
score_R_sym_s

-0.17074893176829298

In [ ]:
# run bootstrap
with open('clean_r_hemisphere_coefficients_wcovariates_MFQ_SCARED_sad_011119.csv', 'w') as csvfile:
    listwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(n_iterations):
        # prepare train and test sets
        train = resample(vals3, n_samples=n_size)
        test = np.array([x for x in vals3 if x.tolist() not in train.tolist()])
        # fit model
        ridge.fit(train[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]], train[:,49])
        # evaluate model
        predictions = ridge.predict(test[:,[24, 26, 28, 30, 32, 36, 38, 2, 44, 54, 55]])
        listwriter.writerow(ridge.coef_)

Significant model with BA9 and BA47 as (negative) significant predictors. No positive predictors when controlling for MFQ/SCARED rather than the internalizing factor score. 

In [ ]:
score_L_sym_s, permuatation_scores_L_sym_s, pvalue_L_sym_s = permutation_test_score(ridge, X_l_sym, y_l_sad, cv=6, scoring = 'neg_mean_squared_error', n_permutations=5000, n_jobs=1)


In [ ]:
pvalue_L_sym_s

Not a significant model in the left hemisphere for sad activation, so I did not run the bootstrapping.